<a href="https://colab.research.google.com/github/monesh1981/GEE_Projectmkb/blob/main/supervisedclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geemap,geemap as map
import ee
ee.Authenticate()
ee.Initialize(project = "ee-moneshbathre")

In [ ]:

m = geemap.Map()
m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
study_area = ee.FeatureCollection('projects/ee-moneshbathre/assets/chennai_bdy')
m.addLayer(study_area,{},"study_area")


m.centerObject(study_area,10)
m

Map(center=[13.100415798205754, 80.23488581056455], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
palette_esacover = ['#006400','	#ffbb22','	#ffff4c','#f096ff','#fa0000','#b4b4b4','#f0f0f0','#0064c8','#0096a0','#00cf75','	#fae6a0']

In [16]:

value_visual = {'min':10,'max':100,'palette':['#006400','#ffbb22','#ffff4c','#f096ff','#fa0000','#b4b4b4','#f0f0f0','#0064c8','#0096a0','	#00cf75','#fae6a0']}
Legend = {'Tree Cover':'#006400',
          'Shrubland':'#ffbb22',
          'Grassland':'#ffff4c',
          'Cropland':'#f096ff',
          'Built-up':'#fa0000',
          'Bare / sparse vegetation':'#b4b4b4',
          'Snow and ice':'#f0f0f0',
          'Permanent water bodies':'#0064c8',
          'Herbaceous wetland':'#0096a0',
          'Mangroves':'#00cf75',
          'Moss and lichen':'#fae6a0'}



In [22]:
esa_worldcover = ee.ImageCollection("ESA/WorldCover/v200").mean().clip(study_area)
m.addLayer(esa_worldcover,value_visual,'esa_worldcover')
m.add_legend(title="My Legend", legend_dict=Legend, **value_visual)

#m.add_legend(legend_title = "ESA WorldCover",legend_dict = legend,position = 'bottomright')
m

Map(bottom=61275.0, center=[13.100879969526488, 80.23590087890626], controls=(WidgetControl(options=['position…

In [23]:
landsat_8 = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate('2021-03-01', '2021-06-30').filterMetadata('CLOUD_COVER','less_than',5).mean().clip(study_area)
vis_params = {'bands': ['B6', 'B5', 'B4'], 'gamma': 1.0, 'min': 5472.0, 'max': 16596.0}
m.addLayer(landsat_8,vis_params,'landst_8')
m

Map(bottom=15585.0, center=[11.985591791400083, 85.06713867187501], controls=(WidgetControl(options=['position…

In [29]:
trg_point = esa_worldcover.sample(**{'region' : study_area,
                                     'scale':20,
                                    'numPixels' : 2000,
                                     'seed' : 0,
                                   'geometries':True})
m.addLayer(trg_point,{},'trg_point')
m

Map(bottom=1440.0, center=[-19.973348786110602, 105.46875000000001], controls=(WidgetControl(options=['positio…

In [30]:
bands = ['B1','B2','B3','B4','B5','B6','B7']
label = 'Map'
training = landsat_8.select(bands).sampleRegions(**{'collection':trg_point,
                                                    'properties':[label],
                                                    'scale':20})

trained = ee.Classifier.smileCart().train(training,label,bands)
result = landsat_8.select(bands).classify(trained)
m.addLayer(result.randomVisualizer(),{},'study_LULC')
m



Map(bottom=486127.0, center=[13.089176149359496, 80.25650796928896], controls=(WidgetControl(options=['positio…